In [2]:
import ccxt
import time
from itertools import permutations
from ccxt.base.errors import RateLimitExceeded

# Replace with your actual API keys
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'

BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_PASSPHRASE = 'BgSecure789Trade2025'  # Required for Bitget authentication

# Connect to Bybit API
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

# Connect to Bitget API
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_PASSPHRASE,
    'options': {'adjustForTimeDifference': True}
})


In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.1 / 100  # 0.1% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
STARTING_CAPITAL = 100  # Configurable capital in USDT

# Trading Fees per Exchange
TRADING_FEES = {
    "Bybit": 0.001,  # 0.1% per trade
    "Bitget": 0.001  # 0.1% per trade
}

def get_tradable_tokens(exchange, exchange_name):
    """Fetches only active spot trading pairs on the given exchange."""
    try:
        markets = exchange.load_markets()
        tradable_tokens = []

        for symbol, market in markets.items():
            # Ensure it's an active SPOT market
            if market.get("type") == "spot" and market.get("active", False):
                if '/USDT' in symbol and ':USDT' not in symbol:  # Ensure it's a valid USDT spot pair
                    try:
                        market_data = exchange.fetch_ticker(symbol)
                        volume = market_data.get('quoteVolume', 0)  # 24h volume
                        if volume and volume >= MIN_VOLUME:
                            tradable_tokens.append(symbol)
                    except RateLimitExceeded:
                        print(f"⚠️ Rate limit hit on {exchange_name}. Pausing...")
                        time.sleep(60)
                        continue
                    except Exception as e:
                        print(f"⚠️ Error fetching market data for {symbol} on {exchange_name}: {e}")
                        continue  # Skip symbols that cause errors

        print(f"✅ Found {len(tradable_tokens)} tradable tokens on {exchange_name}")
        return tradable_tokens

    except Exception as e:
        print(f"❌ Error fetching tradable tokens on {exchange_name}: {e}")
        return []

def get_price(exchange, symbol, exchange_name):
    """Fetch latest price for a given token from the specified exchange."""
    try:
        if exchange_name == "Bybit":
            # Bybit requires a manual request with category='spot'
            symbol_formatted = symbol.replace("/", "")  # Convert "XLM/USDT" -> "XLMUSDT"
            response = exchange.public_get_v5_market_tickers({'symbol': symbol_formatted, 'category': 'spot'})
            return float(response['result']['list'][0]['lastPrice']) if response['result']['list'] else None
        else:
            # Standard fetch_ticker for other exchanges
            if symbol in exchange.load_markets():
                ticker = exchange.fetch_ticker(symbol)
                return float(ticker.get("last", 0)) if ticker else None
        return None
    except RateLimitExceeded:
        print(f"⚠️ Rate limit hit while fetching price from {exchange_name}. Pausing...")
        time.sleep(60)
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol} from {exchange_name}: {e}")
        return None

def log_arbitrage(exchange_name, trade_path, expected_return, net_profit):
    """Logs detected arbitrage trades to a file."""
    with open(f"{exchange_name}_triangular_arbitrage.log", "a") as log_file:
        log_file.write(f"{trade_path} | Expected: {expected_return:.4%} | Net Profit: {net_profit:.4%}\n")

def check_triangular_arbitrage(exchange, exchange_name, tradable_tokens):
    """Checks for triangular arbitrage opportunities within a specific exchange."""
    for route in permutations(tradable_tokens, 3):  # Ensure three different assets
        base_asset = route[0].split('/')[0]  # First asset in the path
        mid_asset = route[1].split('/')[0]
        final_asset = route[2].split('/')[1]  # Final conversion back to USDT

        if base_asset == final_asset:  # Ensure a valid cycle
            try:
                # Get price data for each leg
                first_leg_price = get_price(exchange, route[0], exchange_name)
                second_leg_price = get_price(exchange, route[1], exchange_name)
                third_leg_price = get_price(exchange, route[2], exchange_name)

                # Ensure all prices are valid
                if None in [first_leg_price, second_leg_price, third_leg_price]:
                    continue  

                # Convert using proper exchange path
                amount_after_first_trade = (STARTING_CAPITAL / first_leg_price) * second_leg_price
                amount_after_second_trade = (amount_after_first_trade / second_leg_price) * third_leg_price
                final_amount = amount_after_second_trade

                expected_return = final_amount / STARTING_CAPITAL
                net_expected_return = expected_return * (1 - TRADING_FEES[exchange_name]) ** 3  # Adjust for fees

                # Print all paths for debugging
                print(f"🔍 Checking Arbitrage Path on {exchange_name}: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                print(f"    ✅ Expected Return: {expected_return:.4%}")
                print(f"    ✅ Net Profit After Fees: {net_expected_return:.4%}")

                # Detect and log only profitable trades
                if net_expected_return > 1.001:  # Ensure profit > 0.1%
                    print(f"✅ VALID Triangular Arbitrage Detected on {exchange_name}")
                    print(f"    ✅ Trade Path: {route[0]} ➝ {route[1]} ➝ {route[2]}")
                    print(f"    ✅ Expected Return: {expected_return:.4%}")
                    print(f"    ✅ Net Profit After Fees: {net_expected_return:.4%}")
                    print("-" * 50)
                    log_arbitrage(exchange_name, f"{route[0]} ➝ {route[1]} ➝ {route[2]}", expected_return, net_expected_return)
            except Exception as e:
                print(f"⚠️ Error in arbitrage check on {exchange_name}: {e}")
                continue

while True:
    try:
        # Fetch tradable tokens for each exchange
        bybit_tokens = get_tradable_tokens(bybit, "Bybit")
        bitget_tokens = get_tradable_tokens(bitget, "Bitget")

        if not bybit_tokens and not bitget_tokens:
            print("❌ No tradable tokens found on either exchange. Retrying in 10 seconds...")
            time.sleep(10)
            continue

        print("🔄 Checking for arbitrage opportunities on Bybit and Bitget...")

        # Check arbitrage on Bybit
        if bybit_tokens:
            check_triangular_arbitrage(bybit, "Bybit", bybit_tokens)

        # Check arbitrage on Bitget
        if bitget_tokens:
            check_triangular_arbitrage(bitget, "Bitget", bitget_tokens)

        print("✅ Completed this cycle. Waiting for next check...")
        time.sleep(CHECK_INTERVAL)

    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)  # Avoid infinite crash loops

✅ Found 427 tradable tokens on Bybit
✅ Found 547 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bybit and Bitget...
✅ Completed this cycle. Waiting for next check...
✅ Found 426 tradable tokens on Bybit
✅ Found 547 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bybit and Bitget...
✅ Completed this cycle. Waiting for next check...
✅ Found 426 tradable tokens on Bybit
✅ Found 546 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bybit and Bitget...
✅ Completed this cycle. Waiting for next check...
✅ Found 426 tradable tokens on Bybit
✅ Found 545 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bybit and Bitget...
✅ Completed this cycle. Waiting for next check...
✅ Found 426 tradable tokens on Bybit
✅ Found 543 tradable tokens on Bitget
🔄 Checking for arbitrage opportunities on Bybit and Bitget...
✅ Completed this cycle. Waiting for next check...
✅ Found 426 tradable tokens on Bybit
✅ Found 543 tradable tokens 